In [ ]:
#default_exp build_lib

In [ ]:
#export
from fastcore.utils import *
from fastcore.foundation import *

import pprint
# from json import loads
from jsonref import loads
from collections import namedtuple

## Parse Open API Spec

This library leverages the [OpenAPI Specification](https://github.com/OAI/OpenAPI-Specification) to create a python client for the GitHub API.  The OpenAPI specification contains metadata on all of the endpoints and how to access them properly.  Using this metadata, we can construct a python client dynamically that updates automatically along with the OpenAPI Spec. 

In [ ]:
#export
GH_OPENAPI_URL = 'https://github.com/github/rest-api-description/raw/main/descriptions/api.github.com/api.github.com.json?raw=true'
_DOC_URL = 'https://docs.github.com/'

In [ ]:
# #hide
# s = urlread(GH_OPENAPI_URL)
# js = loads(s)['paths']
# t = js['/repos/{owner}/{repo}/hooks']
# nested_idx(t, *'post requestBody content application/json schema properties'.split())

# t = js['/repos/{owner}/{repo}/hooks']
# params = nested_idx(t, 'get','parameters')
# [o['name'] for o in params if o['in']=='query']

In [ ]:
#export
def build_funcs(nm='ghapi/metadata.py', url=GH_OPENAPI_URL, docurl=_DOC_URL):
    "Build module metadata.py from an Open API spec and optionally filter by a path `pre`"
    def _get_detls(o):
        data = nested_idx(o, *'requestBody content application/json schema properties'.split()) or {}
        url = o['externalDocs']['url'][len(docurl):]
        params = o.get('parameters',None)
        qparams = [p['name'] for p in params if p['in']=='query'] if params else []
        return (o['operationId'], o['summary'], url, qparams, list(data.keys()))
    
    js = loads(urlread(url))
    _funcs = [(path, verb) + _get_detls(detls)
              for path,verbs in js['paths'].items() for verb,detls in verbs.items()
              if 'externalDocs' in detls]
    Path(nm).write_text("funcs = " + pprint.pformat(_funcs, width=360))

In [ ]:
#hide
# build_funcs()

This module created by `build_funcs` contains a list of metadata for each endpoint, containing the path, verb, operation id, summary, documentation relative URL, and list of parameters (if any), e.g:

In [ ]:
#export
GhMeta = namedtuple('GhMeta', 'path verb oper_id summary doc_url params data'.split())

In [ ]:
from ghapi.metadata import funcs
GhMeta(*funcs[3])

GhMeta(path='/app/hook/config', verb='patch', oper_id='apps/update-webhook-config-for-app', summary='Update a webhook configuration for an app', doc_url='v3/apps#update-a-webhook-configuration-for-an-app', params=[], data=['url', 'content_type', 'secret', 'insecure_ssl'])

## Export -

In [ ]:
#hide
from nbdev.export import notebook2script
notebook2script()

Converted 00_core.ipynb.
Converted 90_build_lib.ipynb.
Converted index.ipynb.
